In [1]:
# librerias que utilizaras
import numpy as np
# por si no te lee las tools o functions creadas
import sys
sys.path.append("../")
sys.path.append("../math_tricks/")
# herramientas para leer pdbs
import read_pdb_tools as rpt
# calculo de distancia
from scipy.spatial.distance import pdist
# libreria de tablas
import pandas as pd
# funciones de click generadas en pandas
import funciones_CLICK as fc
# iteradores
import itertools as it
# cuenta tiempo de ejecucion
import datetime

# multiprocessing
import multiprocessing
from functools import partial
# filtro distancia
from scipy.spatial import distance

In [6]:
timenow_bueno = datetime.datetime.now()

timenow = datetime.datetime.now()

# lectura de archivo
file1 = '../pdbs/1xxa_clean.pdb'  # sys.argv[1]
file2 = '../pdbs/1tig_clean.pdb'  # sys.argv[2]

# numero de cliques, preguntar en el software para generalizarlo INPUT...
number_elements_clique = 3

# se define la estructura
pdb1 = rpt.PdbStruct(file1)
pdb2 = rpt.PdbStruct(file2)

# se lee el pdb y se agrega al objeto
pdb1.AddPdbData("%s" % file1)
pdb2.AddPdbData("%s" % file2)

# Se calculan sus vecinos mas cercanos
pdb1.GetNeighbors()
pdb2.GetNeighbors()

# se obtienen los residuos que perteneces a la cadena de interes por default chain = 'A'
pdb11 = pdb1.GetResChain()
pdb22 = pdb2.GetResChain()

# Siempre la proteina 1 es el que se rota y traslada para embonar en la proteina 2
if len(pdb22) < len(pdb11):

    import copy
    pdb1_temp = copy.copy(pdb1)
    pdb2_temp = copy.copy(pdb2)

    pdb11_temp = copy.copy(pdb11)
    pdb22_temp = copy.copy(pdb22)

    pdb1 = pdb2_temp
    pdb2 = pdb1_temp

    pdb11 = pdb22_temp
    pdb22 = pdb11_temp

    del [pdb1_temp]
    del [pdb2_temp]
    del [pdb11_temp]
    del [pdb22_temp]

    print("Intercambio de nombre ya que la proteina 1 es mas grande que la 2")
    print(pdb1.name, len(pdb11))
    print(pdb2.name, len(pdb22))
    print("No te preocupes ya quedo :)")


pdb1.Set_SS()
pdb2.Set_SS()

ss1 = fc.create_ss_table(pdb11)
ss2 = fc.create_ss_table(pdb22)


def get_df_distancias(ref):
    """Funcion para obtener el dataframe de distancias de cada residuo
    Dudas en codigo pueden revisar fc.distancia_entre_atomos en ese se basa
    esta funcion, la diferencia es que se crea con el objeto residuo"""
    # se generan listas con coordenadas y numero de atomo
    coord = [res.GetAtom('CA').coord for res in ref]
    lista_residuos = [res.resi for res in ref]

    # calcula distancia y regresa dataframe
    distancias = [ [pdist(np.array([v, av]), metric='euclidean').item() for av in coord] for v in coord]

    # se genera la matriz de adyacencias para la red
    df_da = pd.DataFrame(index=lista_residuos, columns=lista_residuos, data=distancias)
    return(df_da, lista_residuos)


# devuelve tabla e indices de el dataframe de distancias entre atomos de la misma proteina con dth < 10A
df_distancias1, lista_residuos_1 = get_df_distancias(pdb11)
df_distancias2, lista_residuos2 = get_df_distancias(pdb22)

# se generan cliques, te devuleve dataframe con cliques de k(numero_de_cliques) y la lista de cliques maximales
df_cliques1, cliques1 = fc.gen_3_cliques(df_distancias1,  dth=10, k=number_elements_clique)  # file1[:-4] pal gexf
print('**'*50)
df_cliques2, cliques2 = fc.gen_3_cliques(df_distancias2,  dth=10, k=number_elements_clique)  # file2[:-4]
print('**'*50)

# REVISAR DONDE GUARDA EL GEXF Y COLOCARLO EN LA CARPETA Grafos

# se agrega filtro de residuos que pertenecen a cliques de 7 elementos
mc1_7 = cliques1[cliques1.numero_elementos == 7].drop('numero_elementos', 1)
mc2_7 = cliques2[cliques2.numero_elementos == 7].drop('numero_elementos', 1)

residuos_unicos_1 = []
for i in [list(mc1_7[i].unique()) for i in mc1_7.dropna(1).columns]:
    for j in i:
        residuos_unicos_1.append(int(j))

residuos_unicos_2 = []
for i in [list(mc2_7[i].unique()) for i in mc2_7.dropna(1).columns]:
    for j in i:
        residuos_unicos_2.append(int(j))

residuos_unicos_1 = set(residuos_unicos_1)
residuos_unicos_2 = set(residuos_unicos_2)


# funcion para obtener las propiedades del residuo para los cliques agrupados
def get_df_ca(list_of_residues):
    """Genera dataframe con la informacion necesaria para las siguientes funciones
    FALTA DOCUMENTAR ESTA COSA!!!!"""
    #crear df_ca
    atom_number = []
    atom_name = []
    residue_name = []
    residue_number = []
    coord = []
    res_ngb = []
    for res in list_of_residues:
        for atom in res.atoms:
            atom_number.append(atom.atom_number)
            atom_name.append(atom.name)
            residue_name.append(res.resn)
            residue_number.append(res.resi)
            coord.append(atom.coord)
            res_ngb.append(res.ngb)

    df_atoms = pd.DataFrame(columns=['atom_number', 'atom_name', 'residue_name',
                                   'residue_number', 'vector', 'ngb'])
    df_atoms.atom_number = atom_number
    df_atoms.atom_name = atom_name
    df_atoms.residue_name = residue_name
    df_atoms.residue_number = residue_number
    df_atoms.vector = coord
    df_atoms.ngb = res_ngb

    return(df_atoms)


# CREAR DF_atomos_CA #
df_atoms1 = get_df_ca(pdb11)
df_atoms2 = get_df_ca(pdb22)

numero de cliques maximos encontrados: 158
numero de 3-cliques posibles: 1787
****************************************************************************************************
numero de cliques maximos encontrados: 246
numero de 3-cliques posibles: 2102
****************************************************************************************************


In [7]:
# calculo del RMSD

time = datetime.datetime.now()
print('tiempo generando red:', time - timenow)

print("Se procede a calculo de cliques")

timenow = datetime.datetime.now()

def iter_rmsd(new_df_cliques1,new_df_cliques2,number_elements_clique):
        
    if number_elements_clique == 7:
        print("Filtrando candidatos y preparando datos para alineamiento")
        
    # filtro residuos de 7 unicos
#     for i in new_df_cliques1.columns:
#         mask = np.where(new_df_cliques1[i].isin(residuos_unicos_1), True, False)
#         new_df_cliques1 = new_df_cliques1[mask].reset_index(drop=True)

#     for i in new_df_cliques2.columns:
#         mask = np.where(new_df_cliques2[i].isin(residuos_unicos_2), True, False)
#         new_df_cliques2 = new_df_cliques2[mask].reset_index(drop=True)

    # filtro estructura secundaria
    new_df_cliques1 = fc.paste_SS(ss1, new_df_cliques1, num_cliques=number_elements_clique)
    new_df_cliques2 = fc.paste_SS(ss2, new_df_cliques2, num_cliques=number_elements_clique)
    candidatos_ss = fc.compare_SS(new_df_cliques1, new_df_cliques2, num_cliques=number_elements_clique)

    # rotando y trasladando
    df_cliques1 = fc.get_coords_clique(df_atoms1, new_df_cliques1, number_elements_clique)
    df_cliques2 = fc.get_coords_clique(df_atoms2, new_df_cliques2, number_elements_clique)
    df_cliques1 = fc.baricenter_clique(df_cliques1, number_elements_clique)
    df_cliques2 = fc.baricenter_clique(df_cliques2, number_elements_clique)
    df_cliques1 = fc.center_vectors(df_cliques1, number_elements_clique)
    df_cliques2 = fc.center_vectors(df_cliques2, number_elements_clique)
    idx_rmsd1, idx_rmsd2 = 3 * number_elements_clique, 4 * number_elements_clique + 3 # guardas la posicion de los vectores
    array_df_cliques1 = df_cliques1.values[:, range(idx_rmsd1, idx_rmsd2)]  # del 9 al 15
    array_df_cliques2 = df_cliques2.values[:, range(idx_rmsd1, idx_rmsd2)]

    # filtro de distancia minima
#     df_cliques1 = fc.get_distancia_promedio(number_elements_clique, df_cliques1)
#     df_cliques2 = fc.get_distancia_promedio(number_elements_clique, df_cliques2)
#     array_dist_promedio1 = df_cliques1.values[:, -1]  # el ultimo valor de distancia.
#     array_dist_promedio2 = df_cliques2.values[:, -1]
#     limite_distancia_minima = 0.45
#     if number_elements_clique == 4:
#         limite_distancia_minima = 0.9
#     if number_elements_clique == 5:
#         limite_distancia_minima = 1.8
#     if number_elements_clique == 6:
#         limite_distancia_minima = 3.6
#     if number_elements_clique == 7:
#         limite_distancia_minima = 4.5
#     if number_elements_clique == 8:
#         limite_distancia_minima = 8.0

#     # filtro por distancia minima
#     candidatos_filter_dist = [(candidato_1, candidato_2) for candidato_1, candidato_2 in candidatos_ss if (
#             array_dist_promedio1[candidato_1] - array_dist_promedio2[candidato_2] >= -limite_distancia_minima) & (
#             array_dist_promedio1[candidato_1] - array_dist_promedio2[candidato_1] <= limite_distancia_minima)]
    candidatos_filter_dist = candidatos_ss
    #filtro por restriccion de RMSD despues de ajuste 3D
    p = multiprocessing.Pool(multiprocessing.cpu_count() - 1)
    restriccion_rmsd = 0.15
    if number_elements_clique == 4:
        restriccion_rmsd = 0.30
    if number_elements_clique == 5:
        restriccion_rmsd = 0.60
    if number_elements_clique == 6:
        restriccion_rmsd = 0.90
    if number_elements_clique == 7:
        restriccion_rmsd = 1.50
    if number_elements_clique == 8:
        restriccion_rmsd = 1.80

    rmsd_1 = p.map(partial(fc.calculate_rmsd_rot_trans_m,
                           array_cliques1=array_df_cliques1,
                           array_cliques2=array_df_cliques2,
                           num_cliques=number_elements_clique), candidatos_filter_dist)

    p.close()
    p.join()

    df_candidatos = pd.DataFrame(rmsd_1, columns=['rmsd', 'candidatos', 'matriz_rotacion'])
    df_candidatos = df_candidatos[df_candidatos.rmsd <= restriccion_rmsd]
    df_candidatos['candidato_clique_1'] = df_candidatos.candidatos.str.get(0)
    df_candidatos['candidato_clique_2'] = df_candidatos.candidatos.str.get(1)
    time = datetime.datetime.now()

    print('numero de candidatos:', df_candidatos.shape[0])
    print('tiempo pasado:', time - timenow)
    
#     for i in range(df_candidatos.shape[0]):
#         if df_candidatos.iloc[i].candidato_clique_1 == 190:
#             print(new_df_cliques1.iloc[190])
#     print("////////"*20)
    
#     for i in range(len(new_df_cliques1)):
#         if [float(j) for j in new_df_cliques1[[0,1,2]].values[i]] == [99.0, 100.0, 101.0]:
#             print(new_df_cliques1.iloc[:, :number_elements_clique].values[i])
    
#     for i in range(len(new_df_cliques2)):
#         if [float(j) for j in new_df_cliques2[[0,1,2]].values[i]] == [166.0, 167.0, 168.0]:
#             print(new_df_cliques2.iloc[:, :number_elements_clique].values[i])
                
                
    # Se agrega un nuevo elemento a los cliques.
    if number_elements_clique < 7:
               
        new_df_cliques1 = fc.add_element_clique(df_cliques1, 'candidato_clique_1', cliques1, df_candidatos, number_elements_clique)
        new_df_cliques2 = fc.add_element_clique(df_cliques2, 'candidato_clique_2', cliques2, df_candidatos, number_elements_clique)
        number_elements_clique = number_elements_clique + 1
    

    return(new_df_cliques1, new_df_cliques2, number_elements_clique, df_candidatos)


# primeros cliques a filtrar
new_df_cliques1 = df_cliques1
new_df_cliques2 = df_cliques2

# Si se empieza en 3-cliques solo itera 5 veces para llegar a 7 :v.
for k in range(5):
    new_df_cliques1, new_df_cliques2, number_elements_clique, rmsd = iter_rmsd(
        new_df_cliques1, new_df_cliques2, number_elements_clique)
    print("iteracion", k + 1,"numero de elementos:", number_elements_clique)
    print("===" * 10)
    break

time_all = datetime.datetime.now()
print('tiempo generando cliques candidatos pasando por todos los filtros:', time_all - timenow)

tiempo generando red: 0:00:01.327867
Se procede a calculo de cliques
numero de candidatos: 304
tiempo pasado: 0:01:53.847993
iteracion 1 numero de elementos: 4
tiempo generando cliques candidatos pasando por todos los filtros: 0:01:54.830667


In [9]:
rmsd.head()

,rmsd,candidatos,matriz_rotacion,candidato_clique_1,candidato_clique_2
29757,0.1197,"(33, 363)","[[0.8209012028529064, -0.5187131116664397, 0.23886800317324067], [-0.18432361255716026, 0.15522505868678454, 0.9705307759208681], [-0.540505338556919, -0.840738894628412, 0.03181336276577057]]",33,363
30062,0.0997,"(33, 1018)","[[-0.3101817529803463, 0.8524000047095406, -0.4209530996313182], [0.37870692566717795, 0.5169325121194418, 0.7676990571608108], [0.871991023216268, 0.07870838510005351, -0.4831528180039807]]",33,1018
30125,0.0465,"(33, 1081)","[[0.8693706240154008, 0.4432994689837846, -0.21835818944515784], [0.18019564687767212, 0.12706407177910845, 0.9753892815226396], [0.46013503120867694, -0.887321983533378, 0.030585136798479566]]",33,1081
30214,0.1280,"(33, 1173)","[[0.9597907546227481, 0.22200534521892717, 0.17180027367533615], [0.12448040137856037, -0.8851376456446459, 0.44837035800249464], [0.2516075058763707, -0.4089559572336337, -0.8771822433393154]]",33,1173
30599,0.1277,"(33, 2058)","[[-0.8672109801309065, -0.22072685878490345, 0.4463460202033187], [-0.08891105442516427, 0.9506186818242549, 0.29735323466833385], [-0.4899387108010528, 0.21818289478706854, -0.8440120165500827]]",33,2058


In [95]:
rmsd.reset_index().sample()

,index,rmsd,candidatos,matriz_rotacion,candidato_clique_1,candidato_clique_2
11751,14776,1.1815,"(190, 72)","[[-0.1185001948780261, 0.4660626786053588, -0.8767800655951684], [0.5301586488355432, -0.716911570490311, -0.4527356923879396], [-0.839576983295005, -0.5184818026780823, -0.16213299914867751]]",190,72


In [96]:
clique_in_protein_1 = df_rmsd_1[11751, 3]  # se extrae el indice de los cliques
clique_in_protein_2 = df_rmsd_1[11751, 4]

In [97]:
los_del_clique_1 = df_new_df_clique1[clique_in_protein_1, [0, 1, 2, 3, 4, 5, 6]]
los_del_clique_2 = df_new_df_clique2[clique_in_protein_2, [0, 1, 2, 3, 4, 5, 6]]

In [98]:
los_del_clique_1

array([99.0, 100.0, 101.0, 104.0, 105.0, 108.0, 85.0], dtype=object)

In [99]:
los_del_clique_2

array([166.0, 167.0, 168.0, 169.0, 170.0, 110.0, 113.0], dtype=object)

In [39]:
print("Se procede a alineamiento")

timenow = datetime.datetime.now()
# Obtenemos los elementos para generar el Alineamiento final.
# Utilizando los datos de los cliques, el numero de elementos del clique, la lista de candidatos que sobrevivieron
# y el dataframe del indice de candidatos con el RMSD de cada alineamiento
# quitando el clique seleccionado

df_rmsd = rmsd.reset_index(drop=True)

# se obtiene la matriz de rotacion del menor rmsd
# se aplica a todos los vectores gorro de la proteina 1 que ya se le quito el baricentro del candidato 1
# para cada candidato

protein_to_compare = np.array([vectors for vectors in df_atoms2[df_atoms2.atom_name == 'CA'].vector.values])

atoms_numbers1 = df_atoms1[df_atoms1.atom_name == 'CA'].index.values
atoms_numbers2 = df_atoms2[df_atoms2.atom_name == 'CA'].index.values

df_new_df_clique1 = new_df_cliques1.values
df_new_df_clique2 = new_df_cliques2.values

df_rmsd_1 = df_rmsd.values

vecs1 = np.array([vecs for vecs in df_atoms1.vector.values])
vecs2 = np.array([vecs for vecs in df_atoms2.vector.values])

# baricentro
bari_1 = vecs1.mean(0)
bari_2 = vecs2.mean(0)

# vectores centricos
vecs_center_1 = vecs1 - bari_1
vecs_center_2 = vecs2 - bari_2

def align_residues(idx, so):

    clique_in_protein_1 = df_rmsd_1[idx, 3]  # se extrae el indice de los cliques
    clique_in_protein_2 = df_rmsd_1[idx, 4]

    # se obtienen el numero de residuo de los candidatos
    los_del_clique_1 = df_new_df_clique1[clique_in_protein_1, [0, 1, 2, 3, 4, 5, 6]]

    los_del_clique_2 = df_new_df_clique2[clique_in_protein_2, [0, 1, 2, 3, 4, 5, 6]]

    lista_vectores_gorro = []
    for bari in new_df_cliques1.baricentro_clique.values:
        lista_pre_vectores = [coord - bari for coord in df_atoms1[
            (df_atoms1.atom_name == 'CA') &
            (df_atoms1.residue_number.isin(los_del_clique_1) == False)].vector.values]

        lista_vectores_gorro.append(lista_pre_vectores)

    vectores_gorro_proteina_1 = pd.DataFrame(lista_vectores_gorro)

    df_vectores_gorro_proteina_1 = vectores_gorro_proteina_1.values
    # OJO se extrae por indice no por numero de atomos!!!
    atoms1 = df_atoms1[(df_atoms1.atom_name == 'CA') & (df_atoms1.residue_number.isin(los_del_clique_1) == False)].index.values
    atoms2 = df_atoms2[(df_atoms2.atom_name == 'CA') & (df_atoms2.residue_number.isin(los_del_clique_2) == False)].index.values

    # se guardan los vectores
    protein_to_compare = np.array([vectors for vectors in df_atoms2.iloc[atoms2].vector.values])

    matriz_rotacion = df_rmsd_1[idx, 2]  # se obtiene la matriz de rotacion de los cliques candidatos

    # se obtienen los vectores gorro de la proteina a rotar y trasladar
    vector_gorro = df_vectores_gorro_proteina_1[clique_in_protein_1]  # falta quitar los que no son del clique
    # se aplica la rotacion a vectores gorro
    coord_vectores_rotados = [np.matmul(matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
    # se obtiene el baricentro de la proteina 2
    baricentro_proteina_2 = df_new_df_clique2[clique_in_protein_2, 22]
    # se suma el baricentro a vectores rotados de la proteina 1
    protein_trasladado_rotado = coord_vectores_rotados + baricentro_proteina_2  # nuevas coordendas proteina 1
    # se quitan residuos que pertenezcan al clique candidato para calcular la distancia y emparejar al mejor.

    protein_trasladado_rotado_sin_clique = protein_trasladado_rotado
    protein_to_compare_sin_clique = protein_to_compare

    residuos_match = []  # aqui se guardan las parejas de residuos

    # se itera por cada residuo ya rotado y trasladado

    for residuo_not_in_clique, res_1 in zip(protein_trasladado_rotado_sin_clique, atoms1):
        for residuo, res_2 in zip(protein_to_compare_sin_clique, atoms2):
            distancia = pdist(np.array([residuo_not_in_clique, residuo]), metric='euclidean').item()
            if distancia < 3.5:
                residuos_match.append([distancia, (res_1, res_2)])

    residuos_match = sorted(residuos_match)

    c1 = []
    c2 = []
    cand_n = []
    for i in residuos_match:
        if (i[1][0] in c1) or (i[1][1] in c2) or (i[0] > 3.5):
            continue
        else:
            c1.append(i[1][0])
            c2.append(i[1][1])
            cand_n.append(i)

    parejas = [i[1] for i in cand_n]

    def R_ij(i, j, parejas=parejas):
        valor = sum([vecs_center_1[pos_res[0], i] * vecs_center_2[pos_res[1], j] for pos_res in parejas])
        return valor

    def matrix_R():
        """cliques a comparar: i,j
        desde aqui se itera sobre cada i y hay que variar los vectores
        coordenada
        Regresa la matriz gigante (matriz simetrica del articulo)"""
        # primer renglon
        R11R22R33 = (R_ij(0, 0) + R_ij(1, 1) + R_ij(2, 2))
        R23_R32 = (R_ij(1, 2) - R_ij(2, 1))
        R31_R13 = (R_ij(2, 0) - R_ij(0, 2))
        R12_R21 = (R_ij(0, 1) - R_ij(1, 0))
        # segundo renglon
        R11_R22_R33 = (R_ij(0, 0) - R_ij(1, 1) - R_ij(2, 2))
        R12R21 = (R_ij(0, 1) + R_ij(1, 0))
        R13R31 = (R_ij(0, 2) + R_ij(2, 0))
        # tercer renglon
        _R11R22_R33 = (-R_ij(0, 0) + R_ij(1, 1) - R_ij(2, 2))
        R23R32 = (R_ij(1, 2) + R_ij(2, 1))
        # cuarto renglon
        _R11_R22R33 = (-R_ij(0, 0) - R_ij(1, 1) + R_ij(2, 2))

        matriz_R = [
            [R11R22R33, R23_R32, R31_R13, R12_R21],
            [R23_R32, R11_R22_R33, R12R21, R13R31],
            [R31_R13, R12R21, _R11R22_R33, R23R32],
            [R12_R21, R13R31, R23R32, _R11_R22R33]
        ]
        return (matriz_R)

    def rotation_matrix(matriz_R):
        """utilizando la funcion giant_matrix, fijando los valores de i,j
        se calcula la matriz de rotacion con los eigenvectores y eigenvalores
        arroja una matriz de rotacion que depende de la matriz gigante
        """
        eignvalues, eigenvectors = np.linalg.eig(matriz_R)
        q = eigenvectors[:, np.argmax(eignvalues)]
        q0, q1, q2, q3 = q[0], q[1], q[2], q[3]
        # matriz de rotacion con eigenvectores
        matriz_rotacion = np.array([
            [(q0 ** 2 + q1 ** 2 - q2 ** 2 - q3 ** 2), 2 * (q1 * q2 - q0 * q3), 2 * (q1 * q3 + q0 * q2)],
            [2 * (q1 * q2 + q0 * q3), (q0 ** 2 - q1 ** 2 + q2 ** 2 - q3 ** 2), 2 * (q2 * q3 - q0 * q1)],
            [2 * (q1 * q3 - q0 * q2), 2 * (q2 * q3 + q0 * q1), (q0 ** 2 - q1 ** 2 - q2 ** 2 + q3 ** 2)]
        ], dtype=np.float64)
        return (matriz_rotacion)

    def rotation_vectors(vector_gorro, matriz_rotacion):
        """obtencion de vector rotado,
        utilizando la matriz de rotacion
        y los vectores gorro a rotar y trasladar"""

        coord_rotado = [np.matmul(
            matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
        return (coord_rotado)

    matriz_R = matrix_R()
    matriz_rotacion = rotation_matrix(matriz_R)
    vector_rotado = rotation_vectors(vecs_center_1, matriz_rotacion)
    vector_rotado_trasladado_a_clique2 = vector_rotado + np.array(bari_2)
    protein_trasladado_rotado = vector_rotado_trasladado_a_clique2
    protein_to_compare = vecs2

    residuos_match = []

    for residuo_not_in_clique, res_1 in zip(protein_trasladado_rotado, atoms_numbers1):
        for residuo, res_2 in zip(protein_to_compare, atoms_numbers2):
            distancia = pdist(np.array([residuo_not_in_clique, residuo]), metric='euclidean').item()
            if distancia < 3.5:
                residuos_match.append([distancia, (res_1, res_2)])

#     residuos_match = sorted(residuos_match)

    c1 = []
    c2 = []
    cand_n = []
    for i in residuos_match:
        if (i[1][0] in c1) or (i[1][1] in c2) or (i[0] > 3.5):
            continue
        else:
            c1.append(i[1][0])
            c2.append(i[1][1])
            cand_n.append(i)

    df = pd.DataFrame(cand_n, columns=['distancia', 'candigatos'])

    number_of_residues = df_atoms1[df_atoms1.atom_name =='CA'].shape[0]

    so_temp = len(cand_n) / number_of_residues

    return idx, so_temp, df, matriz_rotacion

timenow_bueno = datetime.datetime.now()

In [39]:
id_so = 0
so = 0.0
df_candidatos = pd.DataFrame()
parejas_clique = []
matriz_rotacion = None
print("Numero candidatos:",len(df_rmsd))
for i in np.arange(len(df_rmsd))[12000:]:
    id_temp, so_temp, df_temp, matriz_rotacion_temp = (align_residues(i, so))
    if so <= so_temp:
        so = so_temp
        id_so = id_temp
        df_candidatos = df_temp
        matriz_rotacion = matriz_rotacion_temp
        print(id_so, so)
    if so == 1:
        break

print('ya termine de alinear residuo a residuo')
print(id_so, so)

Se procede a alineamiento
Numero candidatos: 12754
12000 0.0
12001 0.19718309859154928
12053 0.19718309859154928
12076 0.352112676056338
12112 0.38028169014084506
12142 0.4788732394366197
12143 0.5352112676056338
12181 0.6056338028169014
12210 0.7605633802816901
12218 0.7605633802816901
12219 0.8309859154929577
12429 0.8309859154929577
12471 0.8309859154929577
ya termine de alinear residuo a residuo
12471 0.8309859154929577


In [40]:
df_candidatos.shape[0] / df_atoms1[df_atoms1.atom_name == 'CA'].shape[0]

0.8309859154929577

In [41]:
# new_df_cliques1.to_pickle('clique1.pkl')
# new_df_cliques2.to_pickle('clique2.pkl')
# df_atoms1.to_pickle('clique1_df_atributos.pkl')
# df_atoms2.to_pickle('clique2_df_atributos.pkl')
# df_candidatos.to_pickle('df_alineados.pkl')
# pd.DataFrame(parejas_clique).to_pickle('parejas.pkl')
# df_rmsd.to_pickle('df_rmsd.pkl')

In [43]:
print('Generando rotacion con parejas seleccionadas.')

parejas = [i[1] for i in df_candidatos.values]


def R_ij(i, j, parejas = parejas):
    valor = sum([vecs_center_1[pos_res[0], i] * vecs_center_2[pos_res[1], j] for pos_res in parejas])
    return valor


def matrix_R():
    """cliques a comparar: i,j
    desde aqui se itera sobre cada i y hay que variar los vectores
    coordenada
    Regresa la matriz gigante (matriz simetrica del articulo)"""
    # primer renglon
    R11R22R33 = (R_ij(0, 0) + R_ij(1, 1) + R_ij(2, 2))
    R23_R32 = (R_ij(1, 2) - R_ij(2, 1))
    R31_R13 = (R_ij(2, 0) - R_ij(0, 2))
    R12_R21 = (R_ij(0, 1) - R_ij(1, 0))
    # segundo renglon
    R11_R22_R33 = (R_ij(0, 0) - R_ij(1, 1) - R_ij(2, 2))
    R12R21 = (R_ij(0, 1) + R_ij(1, 0))
    R13R31 = (R_ij(0, 2) + R_ij(2, 0))
    # tercer renglon
    _R11R22_R33 = (-R_ij(0, 0) + R_ij(1, 1) - R_ij(2, 2))
    R23R32 = (R_ij(1, 2) + R_ij(2, 1))
    # cuarto renglon
    _R11_R22R33 = (-R_ij(0, 0) - R_ij(1, 1) + R_ij(2, 2))

    matriz_R = [
        [R11R22R33, R23_R32, R31_R13, R12_R21],
        [R23_R32, R11_R22_R33, R12R21, R13R31],
        [R31_R13, R12R21, _R11R22_R33, R23R32],
        [R12_R21, R13R31, R23R32, _R11_R22R33]
    ]
    return (matriz_R)


def rotation_matrix(matriz_R):
    """utilizando la funcion giant_matrix, fijando los valores de i,j
    se calcula la matriz de rotacion con los eigenvectores y eigenvalores
    arroja una matriz de rotacion que depende de la matriz gigante
    """
    eignvalues, eigenvectors = np.linalg.eig(matriz_R)
    q = eigenvectors[:, np.argmax(eignvalues)]
    q0, q1, q2, q3 = q[0], q[1], q[2], q[3]
    # matriz de rotacion con eigenvectores
    matriz_rotacion = np.array([
        [(q0 ** 2 + q1 ** 2 - q2 ** 2 - q3 ** 2), 2 * (q1 * q2 - q0 * q3), 2 * (q1 * q3 + q0 * q2)],
        [2 * (q1 * q2 + q0 * q3), (q0 ** 2 - q1 ** 2 + q2 ** 2 - q3 ** 2), 2 * (q2 * q3 - q0 * q1)],
        [2 * (q1 * q3 - q0 * q2), 2 * (q2 * q3 + q0 * q1), (q0 ** 2 - q1 ** 2 - q2 ** 2 + q3 ** 2)]
    ], dtype=np.float64)
    return (matriz_rotacion)


def rotation_vectors(vector_gorro, matriz_rotacion):
    """obtencion de vector rotado,
    utilizando la matriz de rotacion
    y los vectores gorro a rotar y trasladar"""

    coord_rotado = [np.matmul(
        matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
    return (coord_rotado)


matriz_R = matrix_R()
matriz_rotacion = rotation_matrix(matriz_R)
vector_rotado = rotation_vectors(vecs_center_1, matriz_rotacion)
vector_rotado_trasladado_a_clique2 = vector_rotado + np.array(bari_2)
protein_trasladado_rotado = vector_rotado_trasladado_a_clique2
protein_to_compare = vecs2

new_df_atom1 = pd.concat([df_atoms1, pd.DataFrame(protein_trasladado_rotado, columns=['x', 'y', 'z'])], 1)
new_df_atom1['new_vector'] = [
    [new_df_atom1.iloc[i]['x'], new_df_atom1.iloc[i]['y'], new_df_atom1.iloc[i]['z']] for i in range(new_df_atom1.shape[0])]

for i in pdb11:
    mask = np.where(i.resi == new_df_atom1.residue_number, True, False)
    for j in new_df_atom1[mask].atom_name:
        mask_2 = np.where(new_df_atom1[mask].atom_name == j, True, False)
        i.GetAtom(j).UpDateValue('coord', new_df_atom1[mask][mask_2].new_vector.values[0])

pdb1.pdbdata = pdb11
pdb1.WriteToFile(file_out_name='1xxa_'+str(datetime.datetime.now()))

time_bueno = datetime.datetime.now()
print('iteraciones completas:', time_bueno - timenow_bueno)
print("archivo nuevo con el nombre de:",'1xxa_'+str(datetime.datetime.now()))

Generando rotacion con parejas seleccionadas.
iteraciones completas: 0:08:56.420122
archivo nuevo con el nombre de: 1xxa_2019-02-13 09:55:15.958870


# Check de como esta alineando los cliques y eligiendolos

In [2]:
# librerias que utilizaras
import numpy as np
# por si no te lee las tools o functions creadas
import sys
sys.path.append("../")
sys.path.append("../math_tricks/")
# herramientas para leer pdbs
import read_pdb_tools as rpt
# calculo de distancia
from scipy.spatial.distance import pdist
# libreria de tablas
import pandas as pd
# funciones de click generadas en pandas
import funciones_CLICK as fc
# iteradores
import itertools as it
# cuenta tiempo de ejecucion
import datetime

# multiprocessing
import multiprocessing
from functools import partial
# filtro distancia
from scipy.spatial import distance

In [3]:
timenow_bueno = datetime.datetime.now()

timenow = datetime.datetime.now()

# lectura de archivo
file1 = '../pdbs/1xxa_clean.pdb'  # sys.argv[1]
file2 = '../pdbs/1tig_clean.pdb'  # sys.argv[2]

# numero de cliques, preguntar en el software para generalizarlo INPUT...
number_elements_clique = 3

# se define la estructura
pdb1 = rpt.PdbStruct(file1)
pdb2 = rpt.PdbStruct(file2)

# se lee el pdb y se agrega al objeto
pdb1.AddPdbData("%s" % file1)
pdb2.AddPdbData("%s" % file2)

# Se calculan sus vecinos mas cercanos
pdb1.GetNeighbors()
pdb2.GetNeighbors()

# se obtienen los residuos que perteneces a la cadena de interes por default chain = 'A'
pdb11 = pdb1.GetResChain()
pdb22 = pdb2.GetResChain()

# Siempre la proteina 1 es el que se rota y traslada para embonar en la proteina 2
if len(pdb22) < len(pdb11):

    import copy
    pdb1_temp = copy.copy(pdb1)
    pdb2_temp = copy.copy(pdb2)

    pdb11_temp = copy.copy(pdb11)
    pdb22_temp = copy.copy(pdb22)

    pdb1 = pdb2_temp
    pdb2 = pdb1_temp

    pdb11 = pdb22_temp
    pdb22 = pdb11_temp

    del [pdb1_temp]
    del [pdb2_temp]
    del [pdb11_temp]
    del [pdb22_temp]

    print("Intercambio de nombre ya que la proteina 1 es mas grande que la 2")
    print(pdb1.name, len(pdb11))
    print(pdb2.name, len(pdb22))
    print("No te preocupes ya quedo :)")


pdb1.Set_SS()
pdb2.Set_SS()

ss1 = fc.create_ss_table(pdb11)
ss2 = fc.create_ss_table(pdb22)


def get_df_distancias(ref):
    """Funcion para obtener el dataframe de distancias de cada residuo
    Dudas en codigo pueden revisar fc.distancia_entre_atomos en ese se basa
    esta funcion, la diferencia es que se crea con el objeto residuo"""
    # se generan listas con coordenadas y numero de atomo
    coord = [res.GetAtom('CA').coord for res in ref]
    lista_residuos = [res.resi for res in ref]

    # calcula distancia y regresa dataframe
    distancias = [ [pdist(np.array([v, av]), metric='euclidean').item() for av in coord] for v in coord]

    # se genera la matriz de adyacencias para la red
    df_da = pd.DataFrame(index=lista_residuos, columns=lista_residuos, data=distancias)
    return(df_da, lista_residuos)


# devuelve tabla e indices de el dataframe de distancias entre atomos de la misma proteina con dth < 10A
df_distancias1, lista_residuos_1 = get_df_distancias(pdb11)
df_distancias2, lista_residuos2 = get_df_distancias(pdb22)

# se generan cliques, te devuleve dataframe con cliques de k(numero_de_cliques) y la lista de cliques maximales
df_cliques1, cliques1 = fc.gen_3_cliques(df_distancias1,  dth=10, k=number_elements_clique)  # file1[:-4] pal gexf
print('**'*50)
df_cliques2, cliques2 = fc.gen_3_cliques(df_distancias2,  dth=10, k=number_elements_clique)  # file2[:-4]
print('**'*50)

# REVISAR DONDE GUARDA EL GEXF Y COLOCARLO EN LA CARPETA Grafos

# se agrega filtro de residuos que pertenecen a cliques de 7 elementos
mc1_7 = cliques1[cliques1.numero_elementos == 7].drop('numero_elementos', 1)
mc2_7 = cliques2[cliques2.numero_elementos == 7].drop('numero_elementos', 1)

residuos_unicos_1 = []
for i in [list(mc1_7[i].unique()) for i in mc1_7.dropna(1).columns]:
    for j in i:
        residuos_unicos_1.append(int(j))

residuos_unicos_2 = []
for i in [list(mc2_7[i].unique()) for i in mc2_7.dropna(1).columns]:
    for j in i:
        residuos_unicos_2.append(int(j))

residuos_unicos_1 = set(residuos_unicos_1)
residuos_unicos_2 = set(residuos_unicos_2)


# funcion para obtener las propiedades del residuo para los cliques agrupados
def get_df_ca(list_of_residues):
    """Genera dataframe con la informacion necesaria para las siguientes funciones
    FALTA DOCUMENTAR ESTA COSA!!!!"""
    #crear df_ca
    atom_number = []
    atom_name = []
    residue_name = []
    residue_number = []
    coord = []
    res_ngb = []
    for res in list_of_residues:
        for atom in res.atoms:
            atom_number.append(atom.atom_number)
            atom_name.append(atom.name)
            residue_name.append(res.resn)
            residue_number.append(res.resi)
            coord.append(atom.coord)
            res_ngb.append(res.ngb)

    df_atoms = pd.DataFrame(columns=['atom_number', 'atom_name', 'residue_name',
                                   'residue_number', 'vector', 'ngb'])
    df_atoms.atom_number = atom_number
    df_atoms.atom_name = atom_name
    df_atoms.residue_name = residue_name
    df_atoms.residue_number = residue_number
    df_atoms.vector = coord
    df_atoms.ngb = res_ngb

    return(df_atoms)


# CREAR DF_atomos_CA #
df_atoms1 = get_df_ca(pdb11)
df_atoms2 = get_df_ca(pdb22)

numero de cliques maximos encontrados: 158
numero de 3-cliques posibles: 1787
****************************************************************************************************
numero de cliques maximos encontrados: 246
numero de 3-cliques posibles: 2102
****************************************************************************************************


In [4]:
df_cliques1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1787 entries, 0 to 1786
Data columns (total 3 columns):
0    1787 non-null int64
1    1787 non-null int64
2    1787 non-null int64
dtypes: int64(3)
memory usage: 42.0 KB


In [5]:
cliques1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 36 to 108
Data columns (total 11 columns):
0                   158 non-null int64
1                   158 non-null int64
2                   158 non-null int64
3                   158 non-null int64
4                   154 non-null float64
5                   142 non-null float64
6                   100 non-null float64
7                   42 non-null float64
8                   16 non-null float64
9                   3 non-null float64
numero_elementos    158 non-null int64
dtypes: float64(6), int64(5)
memory usage: 14.8 KB


In [6]:
df_atoms1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532 entries, 0 to 531
Data columns (total 6 columns):
atom_number       532 non-null int64
atom_name         532 non-null object
residue_name      532 non-null object
residue_number    532 non-null int64
vector            532 non-null object
ngb               532 non-null object
dtypes: int64(2), object(4)
memory usage: 25.0+ KB


In [7]:
def add_element_clique_temp(df_cliques, col_candidatos, cliques, candidatos_df, number_elements_clique):
    cliques_maximales = cliques[
        cliques.numero_elementos >= number_elements_clique].drop('numero_elementos', 1).values
    set_candidatos = [df_cliques.iloc[i, range(
        number_elements_clique)].values for i in candidatos_df[col_candidatos].unique()]
    #conjunto de candidatos unicos
    lista_residuos = []  # aqui se guardara la lista de numero de residuo
    for candidato in set_candidatos:  # este va a cambiar cada iteracion
        for clique_maximal in cliques_maximales:
            clique_maximal = [i for i in clique_maximal if str(i) != 'nan']
            if set(candidato).issubset(clique_maximal):       # si esta en un clique maximal
                no_estan_en_clique = set(clique_maximal).difference(set(candidato))
                # obten los elementos que no estan en ese clique que si estan en el clique maximal
                for nuevo_elemento in no_estan_en_clique:
                    candidato_nuevo = candidato.copy()
                    # se genera una copia para no borrar el orginal
                    candidato_nuevo = np.append(candidato_nuevo, nuevo_elemento)
                    # se apila un elemento de los que no estan
                    if set(candidato_nuevo) not in lista_residuos:
                        lista_residuos.append(set(candidato_nuevo))
                        # si no esta en la lista pasa

    return(pd.DataFrame(lista_residuos))

In [8]:
df_cliques1.head()

,0,1,2
0,96,97,132
1,89,97,132
2,97,98,132
3,97,91,132
4,96,89,132


In [9]:
new_df_cliques1.head()

NameError: name 'new_df_cliques1' is not defined

In [10]:
df_cliques1.head(1)

,0,1,2
0,96,97,132


In [11]:
cliques1.sort_values([0,1,2]).head()

,0,1,2,3,4,5,6,7,8,9,numero_elementos
93,82,83,85,86,84.0000,nan,nan,nan,nan,nan,5
92,82,83,85,86,108.0000,nan,nan,nan,nan,nan,5
95,90,97,88,89,98.0000,99.0000,nan,nan,nan,nan,6
94,90,97,96,99,89.0000,98.0000,nan,nan,nan,nan,6
96,91,96,97,92,94.0000,95.0000,nan,nan,nan,nan,6


In [12]:
df_candidatos.head()

NameError: name 'df_candidatos' is not defined

In [13]:
add_element_clique_temp(df_cliques1, 'candidato_clique_1', cliques1, df_candidatos, number_elements_clique-1).sort_values([0,1,2]).head()

NameError: name 'df_candidatos' is not defined

In [14]:
df_cliques1[[0,1,2]].sort_values([0])

,0,1,2
1015,82,83,86
1016,82,83,108
1017,82,85,86
1014,82,83,85
1018,82,108,85
1019,82,108,86
1025,82,84,86
1024,82,84,85
1023,82,83,84
1013,83,108,86


In [17]:
col_candidatos = 'candidato_clique_1'
#obtengo cliques maximales
cliques_maximales = cliques1[
        cliques1.numero_elementos >= number_elements_clique -1].drop('numero_elementos', 1).values
#obtengo los residuos que se utilizaran para agregarle un elemento
set_candidatos = [df_cliques1.iloc[i, range(
    number_elements_clique -1)].values for i in df_candidatos[col_candidatos].unique()]
#conjunto de candidatos unicos
lista_residuos = []  # aqui se guardara la lista de numero de residuo
for candidato in set_candidatos:  # este va a cambiar cada iteracion
    for clique_maximal in cliques_maximales:
        clique_maximal = [i for i in clique_maximal if str(i) != 'nan']
        if set(candidato).issubset(clique_maximal):       # si esta en un clique maximal
            no_estan_en_clique = set(clique_maximal).difference(set(candidato))
            # obten los elementos que no estan en ese clique que si estan en el clique maximal
            for nuevo_elemento in no_estan_en_clique:
                candidato_nuevo = candidato.copy()
                # se genera una copia para no borrar el orginal
                candidato_nuevo = np.append(candidato_nuevo, nuevo_elemento)
                print(candidato_nuevo)
                # se apila un elemento de los que no estan
                if set(candidato_nuevo) not in lista_residuos:
                    lista_residuos.append(set(candidato_nuevo))
                    # si no esta en la lista pasa

NameError: name 'df_candidatos' is not defined

In [18]:
df_cliques1[[0,1,2]].iloc[12]

0    96
1    97
2    91
Name: 12, dtype: int64

In [19]:
# calculo del RMSD

time = datetime.datetime.now()
print('tiempo generando red:', time - timenow)

print("Se procede a calculo de cliques")

timenow = datetime.datetime.now()

# def iter_rmsd(new_df_cliques1,new_df_cliques2,number_elements_clique):
# primeros cliques a filtrar
new_df_cliques1 = df_cliques1
new_df_cliques2 = df_cliques2

# Si se empieza en 3-cliques solo itera 5 veces para llegar a 7 :v.
for k in range(5):
#     new_df_cliques1, new_df_cliques2, number_elements_clique, rmsd = iter_rmsd(
#         new_df_cliques1, new_df_cliques2, number_elements_clique)
    print("iteracion", k + 1,"numero de elementos:", number_elements_clique)
    
    
    if number_elements_clique == 7:
        print("Filtrando candidatos y preparando datos para alineamiento")
        
    # filtro residuos de 7 unicos
    for i in new_df_cliques1.columns:
        mask = np.where(new_df_cliques1[i].isin(residuos_unicos_1), True, False)
        new_df_cliques1 = new_df_cliques1[mask].reset_index(drop=True)

    for i in new_df_cliques2.columns:
        mask = np.where(new_df_cliques2[i].isin(residuos_unicos_2), True, False)
        new_df_cliques2 = new_df_cliques2[mask].reset_index(drop=True)

    # filtro estructura secundaria
    new_df_cliques1 = fc.paste_SS(ss1, new_df_cliques1, num_cliques=number_elements_clique)
    new_df_cliques2 = fc.paste_SS(ss2, new_df_cliques2, num_cliques=number_elements_clique)
    candidatos_ss = fc.compare_SS(new_df_cliques1, new_df_cliques2, num_cliques=number_elements_clique)

    # rotando y trasladando
    df_cliques1 = fc.get_coords_clique(df_atoms1, new_df_cliques1, number_elements_clique)
    df_cliques2 = fc.get_coords_clique(df_atoms2, new_df_cliques2, number_elements_clique)
    df_cliques1 = fc.baricenter_clique(df_cliques1, number_elements_clique)
    df_cliques2 = fc.baricenter_clique(df_cliques2, number_elements_clique)
    df_cliques1 = fc.center_vectors(df_cliques1, number_elements_clique)
    df_cliques2 = fc.center_vectors(df_cliques2, number_elements_clique)
    idx_rmsd1, idx_rmsd2 = 3 * number_elements_clique, 4 * number_elements_clique + 3 # guardas la posicion de los vectores
    array_df_cliques1 = df_cliques1.values[:, range(idx_rmsd1, idx_rmsd2)]  # del 9 al 15
    array_df_cliques2 = df_cliques2.values[:, range(idx_rmsd1, idx_rmsd2)]

    # filtro de distancia minima
    df_cliques1 = fc.get_distancia_promedio(number_elements_clique, df_cliques1)
    df_cliques2 = fc.get_distancia_promedio(number_elements_clique, df_cliques2)
    array_dist_promedio1 = df_cliques1.values[:, -1]  # el ultimo valor de distancia.
    array_dist_promedio2 = df_cliques2.values[:, -1]
    limite_distancia_minima = 0.45
    if number_elements_clique == 4:
        limite_distancia_minima = 0.9
    if number_elements_clique == 5:
        limite_distancia_minima = 1.8
    if number_elements_clique == 6:
        limite_distancia_minima = 3.6
    if number_elements_clique == 7:
        limite_distancia_minima = 4.5
    if number_elements_clique == 8:
        limite_distancia_minima = 8.0

    # filtro por distancia minima
    candidatos_filter_dist = [(candidato_1, candidato_2) for candidato_1, candidato_2 in candidatos_ss if (
            array_dist_promedio1[candidato_1] - array_dist_promedio2[candidato_2] >= -limite_distancia_minima) & (
            array_dist_promedio1[candidato_1] - array_dist_promedio2[candidato_1] <= limite_distancia_minima)]

    #filtro por restriccion de RMSD despues de ajuste 3D
    p = multiprocessing.Pool(multiprocessing.cpu_count() - 1)
    restriccion_rmsd = 0.15
    if number_elements_clique == 4:
        restriccion_rmsd = 0.30
    if number_elements_clique == 5:
        restriccion_rmsd = 0.60
    if number_elements_clique == 6:
        restriccion_rmsd = 0.90
    if number_elements_clique == 7:
        restriccion_rmsd = 1.50
    if number_elements_clique == 8:
        restriccion_rmsd = 1.80

    rmsd_1 = p.map(partial(fc.calculate_rmsd_rot_trans_m,
                           array_cliques1=array_df_cliques1,
                           array_cliques2=array_df_cliques2,
                           num_cliques=number_elements_clique), candidatos_filter_dist)

    p.close()
    p.join()

    df_candidatos = pd.DataFrame(rmsd_1, columns=['rmsd', 'candidatos', 'matriz_rotacion'])
    df_candidatos = df_candidatos[df_candidatos.rmsd <= restriccion_rmsd]
    df_candidatos['candidato_clique_1'] = df_candidatos.candidatos.str.get(0)
    df_candidatos['candidato_clique_2'] = df_candidatos.candidatos.str.get(1)
    time = datetime.datetime.now()

    print('numero de candidatos:', df_candidatos.shape[0])
    print('tiempo pasado:', time - timenow)
    
#     for i in range(df_candidatos.shape[0]):
#         if df_candidatos.iloc[i].candidato_clique_1 == 190:
#             print(new_df_cliques1.iloc[190])
#     print("////////"*20)
    
#     for i in range(len(new_df_cliques1)):
#         if [float(j) for j in new_df_cliques1[[0,1,2]].values[i]] == [99.0, 100.0, 101.0]:
#             print(new_df_cliques1.iloc[:, :number_elements_clique].values[i])
    
#     for i in range(len(new_df_cliques2)):
#         if [float(j) for j in new_df_cliques2[[0,1,2]].values[i]] == [166.0, 167.0, 168.0]:
#             print(new_df_cliques2.iloc[:, :number_elements_clique].values[i])
                
    pre_df_cliques1 = new_df_cliques1
    pre_df_cliques2 = new_df_cliques2
    pre_df_candidatos = df_candidatos
    # Se agrega un nuevo elemento a los cliques.
    if number_elements_clique < 7:
               
        new_df_cliques1 = fc.add_element_clique(df_cliques1, 'candidato_clique_1', cliques1, df_candidatos, number_elements_clique)
        new_df_cliques2 = fc.add_element_clique(df_cliques2, 'candidato_clique_2', cliques2, df_candidatos, number_elements_clique)
        number_elements_clique = number_elements_clique + 1
    
    print("===" * 10)
#     new_df_cliques1, new_df_cliques2, number_elements_clique, df_candidatos
    break

tiempo generando red: 0:03:53.386626
Se procede a calculo de cliques
iteracion 1 numero de elementos: 3
numero de candidatos: 301
tiempo pasado: 0:00:34.004461


In [43]:
df_candidatos[['rmsd','candidatos']].head()

,rmsd,candidatos
194,0.1023,"(12, 372)"
365,0.1407,"(12, 1039)"
1340,0.1399,"(15, 1398)"
1890,0.1171,"(18, 295)"
3573,0.1294,"(38, 1103)"


In [47]:
pre_df_cliques1.iloc[12]

0                     96                                                                                                                                                                                        
1                     97                                                                                                                                                                                        
2                     91                                                                                                                                                                                        
ss_0                  B                                                                                                                                                                                         
ss_1                  B                                                                                                                                             

In [45]:
pre_df_cliques2.iloc[372][[0,1,2]].head()

0    154
1    156
2    119
Name: 372, dtype: object

for i,j 

In [38]:
idx_rmsd2

19

In [36]:
idx_rmsd1, idx_rmsd2 = 3 * number_elements_clique, 4 * number_elements_clique + 3 # guardas la posicion de los vectores

array_df_cliques1 = df_cliques1.values[:, range(idx_rmsd1, idx_rmsd2)]  # del 9 al 15
array_df_cliques2 = df_cliques2.values[:, range(idx_rmsd1, idx_rmsd2)]

fc.get_rot_vector([1625,997],array_df_cliques1,array_df_cliques2,3)

IndexError: index 16 is out of bounds for axis 1 with size 16

In [32]:
df_cliques1

,0,1,2,ss_0,ss_1,ss_2,coord_clique_0,coord_clique_1,coord_clique_2,matriz_coordenadas,baricentro_clique,vec_gorro_0,vec_gorro_1,vec_gorro_2,vectores_gorro,distancia_promedio
0,96,97,132,B,B,B,"[22.136, 75.657, 3.038]","[21.002, 74.191, 0.793]","[23.896, 66.017, 14.309]","[[22.136, 75.657, 3.038], [21.002, 74.191, 0.793], [23.896, 66.017, 14.309]]","[22.344666666666665, 71.955, 6.046666666666667]","[-0.2086666666666659, 3.701999999999998, -3.008666666666667]","[-1.3426666666666662, 2.236000000000004, -5.253666666666667]","[1.5513333333333357, -5.938000000000002, 8.262333333333332]","[[-0.2086666666666659, 3.701999999999998, -3.008666666666667], [-1.3426666666666662, 2.236000000000004, -5.253666666666667], [1.5513333333333357, -5.938000000000002, 8.262333333333332]]",6.9776
1,89,97,132,B,B,B,"[19.611, 70.025, 3.301]","[21.002, 74.191, 0.793]","[23.896, 66.017, 14.309]","[[19.611, 70.025, 3.301], [21.002, 74.191, 0.793], [23.896, 66.017, 14.309]]","[21.503, 70.07766666666667, 6.134333333333333]","[-1.8919999999999995, -0.052666666666667084, -2.833333333333333]","[-0.5010000000000012, 4.11333333333333, -5.341333333333333]","[2.3930000000000007, -4.060666666666677, 8.174666666666667]","[[-1.8919999999999995, -0.052666666666667084, -2.833333333333333], [-0.5010000000000012, 4.11333333333333, -5.341333333333333], [2.3930000000000007, -4.060666666666677, 8.174666666666667]]",6.5346
2,97,98,132,B,B,B,"[21.002, 74.191, 0.793]","[21.017, 73.308, -0.443]","[23.896, 66.017, 14.309]","[[21.002, 74.191, 0.793], [21.017, 73.308, -0.443], [23.896, 66.017, 14.309]]","[21.971666666666664, 71.17200000000001, 4.886333333333333]","[-0.9696666666666651, 3.0189999999999912, -4.093333333333333]","[-0.9546666666666646, 2.1359999999999957, -5.329333333333333]","[1.9243333333333368, -5.155000000000015, 9.422666666666666]","[[-0.9696666666666651, 3.0189999999999912, -4.093333333333333], [-0.9546666666666646, 2.1359999999999957, -5.329333333333333], [1.9243333333333368, -5.155000000000015, 9.422666666666666]]",7.3033
3,97,91,132,B,B,B,"[21.002, 74.191, 0.793]","[17.937, 70.851, 4.799]","[23.896, 66.017, 14.309]","[[21.002, 74.191, 0.793], [17.937, 70.851, 4.799], [23.896, 66.017, 14.309]]","[20.945, 70.353, 6.633666666666667]","[0.05699999999999861, 3.838000000000008, -5.8406666666666665]","[-3.007999999999999, 0.49800000000000466, -1.8346666666666662]","[2.9510000000000005, -4.3359999999999985, 7.675333333333333]","[[0.05699999999999861, 3.838000000000008, -5.8406666666666665], [-3.007999999999999, 0.49800000000000466, -1.8346666666666662], [2.9510000000000005, -4.3359999999999985, 7.675333333333333]]",6.6146
4,96,89,132,B,B,B,"[22.136, 75.657, 3.038]","[19.611, 70.025, 3.301]","[23.896, 66.017, 14.309]","[[22.136, 75.657, 3.038], [19.611, 70.025, 3.301], [23.896, 66.017, 14.309]]","[21.881, 70.56633333333333, 6.882666666666666]","[0.254999999999999, 5.090666666666664, -3.8446666666666665]","[-2.2699999999999996, -0.541333333333327, -3.581666666666666]","[2.0150000000000006, -4.549333333333337, 7.426333333333333]","[[0.254999999999999, 5.090666666666664, -3.8446666666666665], [-2.2699999999999996, -0.541333333333327, -3.581666666666666], [2.0150000000000006, -4.549333333333337, 7.426333333333333]]",6.5328
5,96,98,132,B,B,B,"[22.136, 75.657, 3.038]","[21.017, 73.308, -0.443]","[23.896, 66.017, 14.309]","[[22.136, 75.657, 3.038], [21.017, 73.308, -0.443], [23.896, 66.017, 14.309]]","[22.349666666666668, 71.66066666666667, 5.634666666666667]","[-0.21366666666666845, 3.9963333333333253, -2.596666666666667]","[-1.3326666666666682, 1.6473333333333358, -6.0776666666666666]","[1.5463333333333331, -5.643666666666675, 8.674333333333333]","[[-0.21366666666666845, 3.9963333333333253, -2.596666666666667], [-1.3326666666666682, 1.6473333333333358, -6.0776666666666666], [1.5463333333333331, -5.643666666666675, 8.674333333333333]]",7.2235
6,96,91,132,B,B,B,"[22.136, 75.657, 3.038]","[17.937, 70.851, 4.799]","[23.896, 66.017, 14.309]","[[22.136, 75.657, 3.038], [17.937, 7

In [39]:
print("Se procede a alineamiento")

timenow = datetime.datetime.now()
# Obtenemos los elementos para generar el Alineamiento final.
# Utilizando los datos de los cliques, el numero de elementos del clique, la lista de candidatos que sobrevivieron
# y el dataframe del indice de candidatos con el RMSD de cada alineamiento
# quitando el clique seleccionado

df_rmsd = rmsd.reset_index(drop=True)

# se obtiene la matriz de rotacion del menor rmsd
# se aplica a todos los vectores gorro de la proteina 1 que ya se le quito el baricentro del candidato 1
# para cada candidato

protein_to_compare = np.array([vectors for vectors in df_atoms2[df_atoms2.atom_name == 'CA'].vector.values])

atoms_numbers1 = df_atoms1[df_atoms1.atom_name == 'CA'].index.values
atoms_numbers2 = df_atoms2[df_atoms2.atom_name == 'CA'].index.values

df_new_df_clique1 = new_df_cliques1.values
df_new_df_clique2 = new_df_cliques2.values

df_rmsd_1 = df_rmsd.values

vecs1 = np.array([vecs for vecs in df_atoms1.vector.values])
vecs2 = np.array([vecs for vecs in df_atoms2.vector.values])

# baricentro
bari_1 = vecs1.mean(0)
bari_2 = vecs2.mean(0)

# vectores centricos
vecs_center_1 = vecs1 - bari_1
vecs_center_2 = vecs2 - bari_2

def align_residues(idx, so):

    clique_in_protein_1 = df_rmsd_1[idx, 3]  # se extrae el indice de los cliques
    clique_in_protein_2 = df_rmsd_1[idx, 4]

    # se obtienen el numero de residuo de los candidatos
    los_del_clique_1 = df_new_df_clique1[clique_in_protein_1, [0, 1, 2, 3, 4, 5, 6]]

    los_del_clique_2 = df_new_df_clique2[clique_in_protein_2, [0, 1, 2, 3, 4, 5, 6]]

    lista_vectores_gorro = []
    for bari in new_df_cliques1.baricentro_clique.values:
        lista_pre_vectores = [coord - bari for coord in df_atoms1[
            (df_atoms1.atom_name == 'CA') &
            (df_atoms1.residue_number.isin(los_del_clique_1) == False)].vector.values]

        lista_vectores_gorro.append(lista_pre_vectores)

    vectores_gorro_proteina_1 = pd.DataFrame(lista_vectores_gorro)

    df_vectores_gorro_proteina_1 = vectores_gorro_proteina_1.values
    # OJO se extrae por indice no por numero de atomos!!!
    atoms1 = df_atoms1[(df_atoms1.atom_name == 'CA') & (df_atoms1.residue_number.isin(los_del_clique_1) == False)].index.values
    atoms2 = df_atoms2[(df_atoms2.atom_name == 'CA') & (df_atoms2.residue_number.isin(los_del_clique_2) == False)].index.values

    # se guardan los vectores
    protein_to_compare = np.array([vectors for vectors in df_atoms2.iloc[atoms2].vector.values])

    matriz_rotacion = df_rmsd_1[idx, 2]  # se obtiene la matriz de rotacion de los cliques candidatos

    # se obtienen los vectores gorro de la proteina a rotar y trasladar
    vector_gorro = df_vectores_gorro_proteina_1[clique_in_protein_1]  # falta quitar los que no son del clique
    # se aplica la rotacion a vectores gorro
    coord_vectores_rotados = [np.matmul(matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
    # se obtiene el baricentro de la proteina 2
    baricentro_proteina_2 = df_new_df_clique2[clique_in_protein_2, 22]
    # se suma el baricentro a vectores rotados de la proteina 1
    protein_trasladado_rotado = coord_vectores_rotados + baricentro_proteina_2  # nuevas coordendas proteina 1
    # se quitan residuos que pertenezcan al clique candidato para calcular la distancia y emparejar al mejor.

    protein_trasladado_rotado_sin_clique = protein_trasladado_rotado
    protein_to_compare_sin_clique = protein_to_compare

    residuos_match = []  # aqui se guardan las parejas de residuos

    # se itera por cada residuo ya rotado y trasladado

    for residuo_not_in_clique, res_1 in zip(protein_trasladado_rotado_sin_clique, atoms1):
        for residuo, res_2 in zip(protein_to_compare_sin_clique, atoms2):
            distancia = pdist(np.array([residuo_not_in_clique, residuo]), metric='euclidean').item()
            if distancia < 3.5:
                residuos_match.append([distancia, (res_1, res_2)])

    residuos_match = sorted(residuos_match)

    c1 = []
    c2 = []
    cand_n = []
    for i in residuos_match:
        if (i[1][0] in c1) or (i[1][1] in c2) or (i[0] > 3.5):
            continue
        else:
            c1.append(i[1][0])
            c2.append(i[1][1])
            cand_n.append(i)

    parejas = [i[1] for i in cand_n]

    def R_ij(i, j, parejas=parejas):
        valor = sum([vecs_center_1[pos_res[0], i] * vecs_center_2[pos_res[1], j] for pos_res in parejas])
        return valor

    def matrix_R():
        """cliques a comparar: i,j
        desde aqui se itera sobre cada i y hay que variar los vectores
        coordenada
        Regresa la matriz gigante (matriz simetrica del articulo)"""
        # primer renglon
        R11R22R33 = (R_ij(0, 0) + R_ij(1, 1) + R_ij(2, 2))
        R23_R32 = (R_ij(1, 2) - R_ij(2, 1))
        R31_R13 = (R_ij(2, 0) - R_ij(0, 2))
        R12_R21 = (R_ij(0, 1) - R_ij(1, 0))
        # segundo renglon
        R11_R22_R33 = (R_ij(0, 0) - R_ij(1, 1) - R_ij(2, 2))
        R12R21 = (R_ij(0, 1) + R_ij(1, 0))
        R13R31 = (R_ij(0, 2) + R_ij(2, 0))
        # tercer renglon
        _R11R22_R33 = (-R_ij(0, 0) + R_ij(1, 1) - R_ij(2, 2))
        R23R32 = (R_ij(1, 2) + R_ij(2, 1))
        # cuarto renglon
        _R11_R22R33 = (-R_ij(0, 0) - R_ij(1, 1) + R_ij(2, 2))

        matriz_R = [
            [R11R22R33, R23_R32, R31_R13, R12_R21],
            [R23_R32, R11_R22_R33, R12R21, R13R31],
            [R31_R13, R12R21, _R11R22_R33, R23R32],
            [R12_R21, R13R31, R23R32, _R11_R22R33]
        ]
        return (matriz_R)

    def rotation_matrix(matriz_R):
        """utilizando la funcion giant_matrix, fijando los valores de i,j
        se calcula la matriz de rotacion con los eigenvectores y eigenvalores
        arroja una matriz de rotacion que depende de la matriz gigante
        """
        eignvalues, eigenvectors = np.linalg.eig(matriz_R)
        q = eigenvectors[:, np.argmax(eignvalues)]
        q0, q1, q2, q3 = q[0], q[1], q[2], q[3]
        # matriz de rotacion con eigenvectores
        matriz_rotacion = np.array([
            [(q0 ** 2 + q1 ** 2 - q2 ** 2 - q3 ** 2), 2 * (q1 * q2 - q0 * q3), 2 * (q1 * q3 + q0 * q2)],
            [2 * (q1 * q2 + q0 * q3), (q0 ** 2 - q1 ** 2 + q2 ** 2 - q3 ** 2), 2 * (q2 * q3 - q0 * q1)],
            [2 * (q1 * q3 - q0 * q2), 2 * (q2 * q3 + q0 * q1), (q0 ** 2 - q1 ** 2 - q2 ** 2 + q3 ** 2)]
        ], dtype=np.float64)
        return (matriz_rotacion)

    def rotation_vectors(vector_gorro, matriz_rotacion):
        """obtencion de vector rotado,
        utilizando la matriz de rotacion
        y los vectores gorro a rotar y trasladar"""

        coord_rotado = [np.matmul(
            matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
        return (coord_rotado)

    matriz_R = matrix_R()
    matriz_rotacion = rotation_matrix(matriz_R)
    vector_rotado = rotation_vectors(vecs_center_1, matriz_rotacion)
    vector_rotado_trasladado_a_clique2 = vector_rotado + np.array(bari_2)
    protein_trasladado_rotado = vector_rotado_trasladado_a_clique2
    protein_to_compare = vecs2

    residuos_match = []

    for residuo_not_in_clique, res_1 in zip(protein_trasladado_rotado, atoms_numbers1):
        for residuo, res_2 in zip(protein_to_compare, atoms_numbers2):
            distancia = pdist(np.array([residuo_not_in_clique, residuo]), metric='euclidean').item()
            if distancia < 3.5:
                residuos_match.append([distancia, (res_1, res_2)])

#     residuos_match = sorted(residuos_match)

    c1 = []
    c2 = []
    cand_n = []
    for i in residuos_match:
        if (i[1][0] in c1) or (i[1][1] in c2) or (i[0] > 3.5):
            continue
        else:
            c1.append(i[1][0])
            c2.append(i[1][1])
            cand_n.append(i)

    df = pd.DataFrame(cand_n, columns=['distancia', 'candigatos'])

    number_of_residues = df_atoms1[df_atoms1.atom_name =='CA'].shape[0]

    so_temp = len(cand_n) / number_of_residues

    return idx, so_temp, df, matriz_rotacion

timenow_bueno = datetime.datetime.now()

In [39]:
id_so = 0
so = 0.0
df_candidatos = pd.DataFrame()
parejas_clique = []
matriz_rotacion = None
print("Numero candidatos:",len(df_rmsd))
for i in np.arange(len(df_rmsd))[12000:]:
    id_temp, so_temp, df_temp, matriz_rotacion_temp = (align_residues(i, so))
    if so <= so_temp:
        so = so_temp
        id_so = id_temp
        df_candidatos = df_temp
        matriz_rotacion = matriz_rotacion_temp
        print(id_so, so)
    if so == 1:
        break

print('ya termine de alinear residuo a residuo')
print(id_so, so)

Se procede a alineamiento
Numero candidatos: 12754
12000 0.0
12001 0.19718309859154928
12053 0.19718309859154928
12076 0.352112676056338
12112 0.38028169014084506
12142 0.4788732394366197
12143 0.5352112676056338
12181 0.6056338028169014
12210 0.7605633802816901
12218 0.7605633802816901
12219 0.8309859154929577
12429 0.8309859154929577
12471 0.8309859154929577
ya termine de alinear residuo a residuo
12471 0.8309859154929577


In [40]:
df_candidatos.shape[0] / df_atoms1[df_atoms1.atom_name == 'CA'].shape[0]

0.8309859154929577

In [41]:
# new_df_cliques1.to_pickle('clique1.pkl')
# new_df_cliques2.to_pickle('clique2.pkl')
# df_atoms1.to_pickle('clique1_df_atributos.pkl')
# df_atoms2.to_pickle('clique2_df_atributos.pkl')
# df_candidatos.to_pickle('df_alineados.pkl')
# pd.DataFrame(parejas_clique).to_pickle('parejas.pkl')
# df_rmsd.to_pickle('df_rmsd.pkl')

In [43]:
print('Generando rotacion con parejas seleccionadas.')

parejas = [i[1] for i in df_candidatos.values]


def R_ij(i, j, parejas = parejas):
    valor = sum([vecs_center_1[pos_res[0], i] * vecs_center_2[pos_res[1], j] for pos_res in parejas])
    return valor


def matrix_R():
    """cliques a comparar: i,j
    desde aqui se itera sobre cada i y hay que variar los vectores
    coordenada
    Regresa la matriz gigante (matriz simetrica del articulo)"""
    # primer renglon
    R11R22R33 = (R_ij(0, 0) + R_ij(1, 1) + R_ij(2, 2))
    R23_R32 = (R_ij(1, 2) - R_ij(2, 1))
    R31_R13 = (R_ij(2, 0) - R_ij(0, 2))
    R12_R21 = (R_ij(0, 1) - R_ij(1, 0))
    # segundo renglon
    R11_R22_R33 = (R_ij(0, 0) - R_ij(1, 1) - R_ij(2, 2))
    R12R21 = (R_ij(0, 1) + R_ij(1, 0))
    R13R31 = (R_ij(0, 2) + R_ij(2, 0))
    # tercer renglon
    _R11R22_R33 = (-R_ij(0, 0) + R_ij(1, 1) - R_ij(2, 2))
    R23R32 = (R_ij(1, 2) + R_ij(2, 1))
    # cuarto renglon
    _R11_R22R33 = (-R_ij(0, 0) - R_ij(1, 1) + R_ij(2, 2))

    matriz_R = [
        [R11R22R33, R23_R32, R31_R13, R12_R21],
        [R23_R32, R11_R22_R33, R12R21, R13R31],
        [R31_R13, R12R21, _R11R22_R33, R23R32],
        [R12_R21, R13R31, R23R32, _R11_R22R33]
    ]
    return (matriz_R)


def rotation_matrix(matriz_R):
    """utilizando la funcion giant_matrix, fijando los valores de i,j
    se calcula la matriz de rotacion con los eigenvectores y eigenvalores
    arroja una matriz de rotacion que depende de la matriz gigante
    """
    eignvalues, eigenvectors = np.linalg.eig(matriz_R)
    q = eigenvectors[:, np.argmax(eignvalues)]
    q0, q1, q2, q3 = q[0], q[1], q[2], q[3]
    # matriz de rotacion con eigenvectores
    matriz_rotacion = np.array([
        [(q0 ** 2 + q1 ** 2 - q2 ** 2 - q3 ** 2), 2 * (q1 * q2 - q0 * q3), 2 * (q1 * q3 + q0 * q2)],
        [2 * (q1 * q2 + q0 * q3), (q0 ** 2 - q1 ** 2 + q2 ** 2 - q3 ** 2), 2 * (q2 * q3 - q0 * q1)],
        [2 * (q1 * q3 - q0 * q2), 2 * (q2 * q3 + q0 * q1), (q0 ** 2 - q1 ** 2 - q2 ** 2 + q3 ** 2)]
    ], dtype=np.float64)
    return (matriz_rotacion)


def rotation_vectors(vector_gorro, matriz_rotacion):
    """obtencion de vector rotado,
    utilizando la matriz de rotacion
    y los vectores gorro a rotar y trasladar"""

    coord_rotado = [np.matmul(
        matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
    return (coord_rotado)


matriz_R = matrix_R()
matriz_rotacion = rotation_matrix(matriz_R)
vector_rotado = rotation_vectors(vecs_center_1, matriz_rotacion)
vector_rotado_trasladado_a_clique2 = vector_rotado + np.array(bari_2)
protein_trasladado_rotado = vector_rotado_trasladado_a_clique2
protein_to_compare = vecs2

new_df_atom1 = pd.concat([df_atoms1, pd.DataFrame(protein_trasladado_rotado, columns=['x', 'y', 'z'])], 1)
new_df_atom1['new_vector'] = [
    [new_df_atom1.iloc[i]['x'], new_df_atom1.iloc[i]['y'], new_df_atom1.iloc[i]['z']] for i in range(new_df_atom1.shape[0])]

for i in pdb11:
    mask = np.where(i.resi == new_df_atom1.residue_number, True, False)
    for j in new_df_atom1[mask].atom_name:
        mask_2 = np.where(new_df_atom1[mask].atom_name == j, True, False)
        i.GetAtom(j).UpDateValue('coord', new_df_atom1[mask][mask_2].new_vector.values[0])

pdb1.pdbdata = pdb11
pdb1.WriteToFile(file_out_name='1xxa_'+str(datetime.datetime.now()))

time_bueno = datetime.datetime.now()
print('iteraciones completas:', time_bueno - timenow_bueno)
print("archivo nuevo con el nombre de:",'1xxa_'+str(datetime.datetime.now()))

Generando rotacion con parejas seleccionadas.
iteraciones completas: 0:08:56.420122
archivo nuevo con el nombre de: 1xxa_2019-02-13 09:55:15.958870
